In [ ]:
import pathlib
import shutil

import cv2
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
ref_img_paths = [
    "..\data\\raw\\uzhref\\sun9_00123.npy",
    "..\data\\raw\\uzhref\\sun2_00972.npy",
    "..\data\\raw\\uzhref\\sun2_01050.npy",
    "..\data\\raw\\uzhref\\sun5_00723.npy",
    "..\data\\raw\\uzhref\\sun9_00288.npy",
    "..\data\\raw\\uzhref\\sun5_00288.npy",
    "..\data\\raw\\uzhref\\sun2_01227.npy",
    "..\data\\excluded\\uzh\\highway1_02661.npy",
]

In [ ]:
ref_images = [np.load(p) for p in ref_img_paths]

In [ ]:
target_path = pathlib.Path("..\\data\\ref")

In [ ]:
for img_path, img in zip(ref_img_paths, ref_images):
    img_rgb = img[:, :, [2, 1, 0]]
    event = img[:, :, 3]
    mask = img[:, :, 4]
    # img_gs = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    img_hsl = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HLS)
    brightness = img_hsl[:, :, 1]

    diff = cv2.absdiff(brightness, event)
    diff[mask == 0] = 0
    diff_mask = (diff > 110).astype(np.uint8) * 255
    # close diff mask
    kernel = np.ones((5, 5), np.uint8)
    diff_mask = cv2.morphologyEx(diff_mask, cv2.MORPH_CLOSE, kernel)
    # erode 2 times
    diff_mask = cv2.erode(diff_mask, kernel, iterations=2)
    # dilate 5 times
    diff_mask = cv2.dilate(diff_mask, kernel, iterations=25)
    diff_mask[mask == 0] = 0

    img_rgb_cp = img_rgb.copy()
    img_rgb_cp[diff_mask == 255] = 0
    img_path = pathlib.Path(img_path)
    stacked = np.dstack([img_rgb_cp, event, diff_mask])
    np.save(target_path / img_path.name, stacked)